# **Imports and defining variables**

In [1]:
import pandas as pd
import re
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [2]:
navigator = 'Microsoft Edge (Scrapping; This is my system)'
url_allbrands_homepage = 'https://www.auto-data.net/en/allbrands'
url = 'https://www.auto-data.net'

In [3]:
html = requests.get(url_allbrands_homepage, headers={'User-Agent': navigator})
html

<Response [200]>

# **Scraping**

## Creating the soup and cleaning the information to get only text

In [4]:
soup = BeautifulSoup(html.text, 'html.parser')

In [5]:
brands = soup.find_all('a', {'class' : 'marki_blok'})
brands[0:5]

[<a class="marki_blok" href="/en/abarth-brand-200" title="Abarth - Technical Specs, Fuel consumption, Dimensions"><img alt="Abarth - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/Abarth.png"/><strong>Abarth</strong></a>,
 <a class="marki_blok" href="/en/ac-brand-1" title="AC - Technical Specs, Fuel consumption, Dimensions"><img alt="AC - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/AC.png"/><strong>AC</strong></a>,
 <a class="marki_blok" href="/en/acura-brand-6" title="Acura - Technical Specs, Fuel consumption, Dimensions"><img alt="Acura - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/Acura.png"/><strong>Acura</strong></a>,
 <a class="marki_blok" href="/en/aiways-brand-301" title="Aiways - Technical Specs, Fuel consumption, Dimensions"><img alt="Aiways - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/Aiways.png"/><strong>Aiways</strong></a>,
 <a class="marki_blok" href="/en/aixam-brand-255" title="Aixam - Technical Sp

In [6]:
#This loop is to get only the part to use in link ie whatever is after 'href='
brand_list = re.findall('href="(\S+)"',str(brands))
brand_list[0:5]

['/en/abarth-brand-200',
 '/en/ac-brand-1',
 '/en/acura-brand-6',
 '/en/aiways-brand-301',
 '/en/aixam-brand-255']

In [57]:
#Looping to remove the /en since the url already has it and removing it makes it so it will default to another language
brand_list_final = []
for i in range(len(brand_list)):
    brand_list_final += re.findall('en/(\S+)', brand_list[i])
brand_list_final[0:5]
#Starting to create dataframe for the links
df = pd.DataFrame(brand_list_final)
df['letter'] = df[0].apply(lambda x: x[0])
df['letter'].unique()

array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'z'],
      dtype=object)

In [73]:
#Creating CSV files for each starting letter of the brand names so we can save it more often
dicto = {}
for letter in df['letter'].unique():
    a = df[df['letter'] == letter].drop(columns = 'letter')
    a.to_csv('../BEEV/letter_csvs/' + letter + '.csv')
    dicto[letter] = a

## Creating the loops for the actual scraping

In [75]:
url = 'https://www.auto-data.net/en/'
for csvs in dicto.values():
    csvs[0] = csvs[0].apply(lambda x: url + x)

In [78]:
#Dataframe with url for each brand
for letter, letter_csvs in dicto.items():
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [ ]:
# #Looping to get the actual url for each brand
# urls_list = []
# for urls in brand_list:
#     url = 'https://www.auto-data.net'
#     urls_list += [url + urls]
# urls_list[0:5]

In [99]:
#Function to get the models inside each brand
def model_of_each_brand(brand_url):
    models_list = []
    html2 = requests.get(brand_url, headers={'User-Agent': navigator})
    soup2 = BeautifulSoup(html2.text, 'html.parser')
    models = soup2.find_all('a', {'class' : 'modeli'}) 
    models_list_final = re.findall('en/(\S+)', str(models))
    return [x[:-1] for x in models_list_final]
brand_url = 'https://www.auto-data.net/en/abarth-brand-200'
model_of_each_brand(brand_url)

['abarth-124-spider-model-2152',
 'abarth-500-model-2872',
 'abarth-595-model-2149',
 'abarth-695-model-2150']

In [123]:
#Applying the function to get add the url to the models
for csvs in dicto.values():
    csvs['model_url'] = csvs[0].apply(model_of_each_brand)

KeyError: 'model_url'

In [ ]:
###Looping for the correct generation/series in each model
series_list = []
series_list_final = []
for page3 in urls_list2:
    # time.sleep(60 * random.random.uniform(1-10))
    html3 = requests.get(page3, headers={'User-Agent': navigator})
    soup3 = BeautifulSoup(html3.text, 'html.parser')
    series = soup3.find_all('a', {'class' : 'position'}) 
    for serie in series:
        # print('\n')
        # print(serie)
        varible = re.findall('href="(\S+)"',str(serie))
        # print(varible)
        series_list_final += varible

In [51]:
var_list = pd.DataFrame(series_list_final).drop_duplicates().reset_index().drop(columns = ['index'])
var_list.to_csv('series_list_final_2.csv', header = False, index = False)

In [ ]:
#Creating another urls list for the final part
# urls_list3 = []
# for urls3 in var_list['0']:
    # url = 'https://www.auto-data.net'
#     urls_list3 += [url + "/" +urls3]

# urls_list3
var_list['url3'] = var_list['0'].apply(lambda x: 'https://www.auto-data.net' + x)

In [ ]:
#This is the loop for the modifications(engines, facelifts, etc)
for page4 in var_list['url3']:
    html4 = requests.get(page4, headers={'User-Agent': navigator})
    soup4 = BeautifulSoup(html4.text, 'html.parser')
    mods = soup4.find_all('a', {'rel' : 'nofollow'})           #It's working!!
     

In [ ]:
#This is the final loop to get the details for the cars
specs = soup4.find_all('table', {'class' : 'cardetailsout car2'})

In [ ]:
#Need to install this library to work
!pip install html5lib

## Final dataframe

In [96]:
#This is the dataframe with all the information we want
page4 = 'https://www.auto-data.net/en/alfa-romeo-mito-1.6-jtdm-120hp-16680'
what = pd.read_html(page4)[1]  
what

what2 = what[(what['General information'] == 'Brand')
|(what['General information'] == 'Model')
|(what['General information'] == 'Generation')
|(what['General information'] == 'Modification (Engine)')
|(what['General information'] == 'CO2 emissions') 
|(what['General information'] == 'Body type')
|(what['General information'] == 'Fuel tank capacity')
|(what['General information'] == 'Fuel consumption (economy) - combined')
|(what['General information'] == 'Fuel Type')
|(what['General information'] == 'Power')]
what2.index = what2['General information']
what3 = what2.T
what3.drop(axis = 0, labels = 'General information', inplace = True)
what3['Fuel tank(l)'] = what3['Fuel tank capacity'].apply(lambda x : x.split(' ')[0])
what3['Fuel consumption(l/100km)'] = what3['Fuel consumption (economy) - combined'].apply(lambda x : x.split(' ')[0])
what3['Power(Hp)'] = what3['Power'].apply(lambda x: x.split(' ')[0])
what3['Range (Km)'] = what3[['Fuel tank(l)','Fuel consumption(l/100km)']].apply(lambda item: int(item[0])/float(item[1])*100,axis=1)

what4 = what3
what4.drop(columns = ['Fuel consumption (economy) - combined', 'Fuel tank capacity'], inplace = True)
what4                               #We need to remove this General information column still

General information,Brand,Model,Generation,Modification (Engine),Body type,CO2 emissions,Fuel Type,Power,Fuel tank(l),Fuel consumption(l/100km),Power(Hp),Range (Km)
General information.1,Alfa Romeo,MiTo,MiTo,1.6 JTDm (120 Hp),Hatchback,122 g/km,Diesel,120 Hp @ 3750 rpm.,45,4.8,120,937.5
